In [1]:
#from nbdev.showdoc import *

#export
import numpy as np
#import quantecon as qe

import matplotlib.pyplot as plt

from numba import vectorize
from numba import jit, njit, prange

from numba import int32, float32, double, boolean    # import the types
from numba.experimental import jitclass

from scipy import stats
from scipy.stats import poisson
from scipy.stats import nbinom
from scipy.optimize import minimize

import time
import math
import csv

In [2]:
#export

@njit
def rand_choice_nb(arr, prob, seed):
    set_numba_numpy_random_seed(seed)
    """
    :param arr: A 1D numpy array of values to sample from.
    :param prob: A 1D numpy array of probabilities for the given samples.
    :return: A random sample from the given array with a given probability.
    """
    return arr[np.searchsorted(np.cumsum(prob), np.random.random(), side="right")]

In [3]:
#export

@njit
def set_numba_numpy_random_seed(seed):
    np.random.seed(seed)

def set_numpy_random_seed(seed):
    np.random.seed(seed)
    set_numba_numpy_random_seed(seed)
    

In [4]:
#export

@njit
def inverse_transform_binomial(n,p,u):    
    c= p/(1-p)    
    #print (c)
    i = 0
    pr = (1-p)**n    
    F = pr
    
    while i < n+1:        
        if u <= F:
            return i          
        pr = ( (c*(n-i)) / (i+1) ) * pr        
        F = F+pr    
        i+=1            
    return i-1

In [61]:
p = 0.1
u = 0.8

n_avg = 100
percent_n_avg = inverse_transform_binomial(n_avg,p,u) / n_avg


print (percent_n_avg)

for n in range(1,50,5):
    value = inverse_transform_binomial(n,p,u)        
    approx_value = round(percent_n_avg * n - 0.5)
    abs_error = value - approx_value
    print(n, value , approx_value, abs_error)


0.12
1 0 0 0
6 1 0 1
11 2 1 1
16 3 1 2
21 3 2 1
26 4 3 1
31 4 3 1
36 5 4 1
41 6 4 2
46 6 5 1


In [6]:
#export

#@njit
def generate_demand_data(number_of_simulation_periods, number_of_optimisation_paths, lambda_mu, lambda_omega, seed, demand_is_deterministic=False):
        
    demand_evaluation = np.full(number_of_simulation_periods, 0)
    demand_optimisation = np.full((number_of_optimisation_paths, number_of_simulation_periods), 0)

    for i in range(number_of_simulation_periods):
        set_numpy_random_seed(seed*(i+1))
        mu =  np.random.poisson(lambda_mu)
        omega = np.random.poisson(lambda_omega)
        sigma = np.sqrt(mu + omega)

        r0 = mu*mu / (sigma*sigma - mu)
        p = (sigma*sigma - mu) / (sigma*sigma)
    
        set_numpy_random_seed(seed*(i+1))
        demand_evaluation[i] = np.random.negative_binomial(r0, 1-p)
    
        if demand_is_deterministic == False:
            set_numpy_random_seed(seed*i+1000)        
            demand_optimisation[:,i] = np.random.negative_binomial(r0, 1-p, number_of_optimisation_paths)
        
        else:
            demand_optimisation[:,i] = np.full(number_of_optimisation_paths, mu)

    return demand_optimisation, demand_evaluation

In [7]:
#export

#@njit
def generate_supply_data_evaluation(number_of_simulation_periods, pi, beta1, beta2, type_last_period, seed):

    set_numpy_random_seed(seed)
    fail_type_current = rand_choice_nb(np.arange(3), pi[type_last_period:,], seed)
    if fail_type_current < 2:
        relative_shortage_evaluation = fail_type_current

    else:
        relative_shortage_evaluation = np.random.beta(beta1, beta2)

    for i in range(1, number_of_simulation_periods):
        set_numpy_random_seed(seed+i)
        fail_type_current = rand_choice_nb(np.arange(3), pi[fail_type_current:,], seed)
    
        if fail_type_current < 2:
            relative_shortage_evaluation = np.append(relative_shortage_evaluation, fail_type_current)

        else:
            set_numpy_random_seed(seed+10*i+100)
            relative_shortage_evaluation = np.append(relative_shortage_evaluation, np.random.beta(beta1, beta2))
    
    return relative_shortage_evaluation


In [8]:
#export

@njit
def generate_supply_data_optimisation(pi, pi_stat, beta1, beta2, number_of_optimisation_paths, tau, nu, type_last_period, seed, supply_is_deterministic=False):

    if supply_is_deterministic == False:
        relative_shortage_optimisation = np.full((number_of_optimisation_paths, tau+nu+1), 0, dtype = np.float32)
        
        for i in range(number_of_optimisation_paths):
            set_numba_numpy_random_seed(seed+i)
            
            fail_type_current = rand_choice_nb(np.arange(3), pi[type_last_period:,], seed)
            
            relative_shortage = np.empty(tau+nu+1, dtype = np.float32)
        
            if fail_type_current < 2:
                relative_shortage[0] = fail_type_current

            else:
                relative_shortage[0] = np.random.beta(beta1, beta2)
            
            if tau+nu > 0:
                for j in range(tau+nu):
                    fail_type_current = rand_choice_nb(np.arange(3), pi[fail_type_current:,], seed)
    
                    if fail_type_current < 2:
                        relative_shortage[j+1] = fail_type_current

                    else:
                        relative_shortage[j+1] = np.random.beta(beta1, beta2)
                
            relative_shortage_optimisation[i,:] = relative_shortage
                
    else:
        relative_shortage_optimisation = np.full((number_of_optimisation_paths, tau+nu+1), pi_stat[1]+pi_stat[2]*beta1/(beta1+beta2), dtype=np.float32)

    return relative_shortage_optimisation


In [9]:
#export

#@njit
def generate_spoilage_data(spoil_probabilities, number_of_optimisation_paths, number_of_simulation_periods, seed, spoilage_is_deterministic=False):
    
    max_number_of_spoil = len(spoil_probabilities)
    set_numpy_random_seed(seed)
    spoilage_data_evaluation = np.array(np.zeros([max_number_of_spoil, number_of_simulation_periods]))
    spoilage_data_optimisation = np.array(np.zeros([max_number_of_spoil, number_of_optimisation_paths, number_of_simulation_periods]))
    for i in range(max_number_of_spoil):
        set_numpy_random_seed(seed+30*i)
        spoilage_data_evaluation[i,:] = np.random.uniform(0, 1, number_of_simulation_periods)
        if spoilage_is_deterministic == False:
            for j in range(number_of_optimisation_paths):
                set_numpy_random_seed(seed+20*i+10*j+100)
                spoilage_data_optimisation[i,j,:] = np.random.uniform(0, 1, number_of_simulation_periods)
    
    if spoilage_is_deterministic == True:
        spoilage_data_optimisation = np.zeros([max_number_of_spoil, number_of_optimisation_paths, number_of_simulation_periods])
        foo = 0
        for i in range(max_number_of_spoil):
            foo += (i+1)*spoil_probabilities[i]
        
        expec = np.int8(np.around(foo))
            
        spoilage_data_optimisation[expec-1,:,:] = 1
        
    return spoilage_data_optimisation, spoilage_data_evaluation


In [26]:
#export

@njit
def add_supply(supply, order):
    if order < 0:
        print ("order error")
        return 0
    else:
        return  np.round((1 - supply) * order)



In [11]:
#export

@njit
def take_out_demand(inventory, demand):
    total_inventory = np.sum(inventory)
    underage = max(demand - total_inventory, 0)

    if underage > 0:
        inventory.fill(0)

    else:
        remaining_demand = demand
    
        for row in range(inventory.shape[0]-1,-1 ,-1):  
            if remaining_demand >= inventory[row]:    
                remaining_demand -= inventory[row]
                inventory[row] = 0
            elif remaining_demand > 0:
                inventory[row] -= remaining_demand
                if inventory[row] < 0:
                    print ("Something wrong here: Negative Inventory")
                remaining_demand = 0
            
    return inventory, underage
        

In [12]:
#export

@njit
def get_spoilage(spoil_probabilities, inventory, spoilage_evaluation):

    spoilage_random = spoilage_evaluation

    max_number_of_spoil = len(spoil_probabilities)
    spoil_probability_period = np.zeros(max_number_of_spoil)
    spoilage = np.zeros(max_number_of_spoil)
    
    for period in range(0, max_number_of_spoil-1):
        spoil_probability_period[period] = spoil_probabilities[period]/(1-np.sum(spoil_probabilities[:period]))
        
    spoil_probability_period[max_number_of_spoil-1] = 1


    for i in range(max_number_of_spoil-1):
        spoilage[i] = inverse_transform_binomial(inventory[i], spoil_probability_period[i], spoilage_random[i])
        inventory[i] -= spoilage[i]

    spoilage[max_number_of_spoil-1] = inventory[max_number_of_spoil-1]
    inventory[max_number_of_spoil-1] = 0

    spoilage_sum = np.sum(spoilage)

    return inventory, spoilage_sum

In [12]:
#export

@njit
def get_spoilage(spoil_probabilities, inventory, spoilage_evaluation):

    n_avg = 20
    percent_n_avg = inverse_transform_binomial(n_avg,p,u) / n_avg
    
    spoilage_random = spoilage_evaluation

    max_number_of_spoil = len(spoil_probabilities)
    spoil_probability_period = np.zeros(max_number_of_spoil)
    spoilage = np.zeros(max_number_of_spoil)
    
    for period in range(0, max_number_of_spoil-1):
        spoil_probability_period[period] = spoil_probabilities[period]/(1-np.sum(spoil_probabilities[:period]))
        
    spoil_probability_period[max_number_of_spoil-1] = 1


    for i in range(max_number_of_spoil-1):
        spoilage[i] = inverse_transform_binomial(inventory[i], spoil_probability_period[i], spoilage_random[i])
        inventory[i] -= spoilage[i]

    spoilage[max_number_of_spoil-1] = inventory[max_number_of_spoil-1]
    inventory[max_number_of_spoil-1] = 0

    spoilage_sum = np.sum(spoilage)

    return inventory, spoilage_sum

In [13]:
#export

@njit
def transfer_inventory(inventory):
    inventory_next_period = np.roll(inventory, 1)    
    inventory_next_period[0] = 0
    
    return inventory_next_period

In [34]:
#export

@njit
def period_transition(inventory, supply_evaluation_period, orders_period, demand_evaluation_period, spoil_probabilities, spoilage_evaluation_period, h, v, b):
    cost = 0
    inventory[0] = add_supply(supply_evaluation_period, orders_period)
    inventory, underage = take_out_demand(inventory, demand_evaluation_period)
    cost += underage*b
    inventory, spoilage = get_spoilage(spoil_probabilities, inventory, spoilage_evaluation_period)
    cost += spoilage*h + np.sum(inventory)*v
    inventory = transfer_inventory(inventory)
    
    return inventory, cost

In [33]:
@njit
def calculate_starting_inventory(number_of_optimisation_paths, init_inventory, supply_optimisation_path, orders_path, demand_optimisation_path, spoil_probabilities, spoilage_optimisation_path, tau, h, v, b):
    inventory_state = np.full((number_of_optimisation_paths, len(spoil_probabilities)), 0)
    for i in range(number_of_optimisation_paths):
        inventory = init_inventory.copy()
        for j in range(tau):
            inventory, cost = period_transition(inventory, supply_optimisation_path[i,j], orders_path[j], demand_optimisation_path[i,j], spoil_probabilities, spoilage_optimisation_path[:,i,j], h, v, b)
        inventory_state[i,:] = inventory

    return inventory_state

In [16]:
@njit
def calculate_order_costs(theta_star, number_of_optimisation_paths, inventory_start, supply_optimisation_path, demand_optimisation_path, spoil_probabilities, spoilage_optimisation_path, nu, rho, h, v, b):
    path_cost = np.zeros(number_of_optimisation_paths, dtype = np.float32)
    inventory_paths = inventory_start.copy()
    for i in range(number_of_optimisation_paths):
        inventory = inventory_paths[i,:]
        total_cost = 0
        for j in range(nu+1):
            inventory, cost = period_transition(inventory, supply_optimisation_path[i,j], np.round(np.exp(theta_star[j])), demand_optimisation_path[i,j], spoil_probabilities, spoilage_optimisation_path[:,i,j], h, v, b)
            total_cost += cost*rho**j
        path_cost[i] = total_cost
    
    return np.mean(path_cost)

In [17]:
def optimise_order_decision(new_orders, number_of_optimisation_paths, init_inventory, supply_optimisation_paths, demand_optimisation_paths, spoilage_optimisation_paths, spoil_probabilities, order_paths, tau, nu, rho, h, v, b):
    inventory_start = calculate_starting_inventory(number_of_optimisation_paths, init_inventory, supply_optimisation_paths[:,:tau], order_paths, demand_optimisation_paths[:,:tau], spoil_probabilities, spoilage_optimisation_paths[:,:,:tau], tau, h, v, b)
    theta_star = np.log(new_orders)    
    res = minimize(calculate_order_costs, x0=theta_star, args=(number_of_optimisation_paths, inventory_start, supply_optimisation_paths[:,tau:], demand_optimisation_paths[:,tau:], spoil_probabilities, spoilage_optimisation_paths[:,:,tau:], nu, rho, h, v, b), method='Nelder-Mead', tol=1e-9)
        
    return res.x[0]

In [29]:
def optimise_order_decision(new_orders, number_of_optimisation_paths, init_inventory, supply_optimisation_paths, demand_optimisation_paths, spoilage_optimisation_paths, spoil_probabilities, order_paths, tau, nu, rho, h, v, b):
    inventory_start = calculate_starting_inventory(number_of_optimisation_paths, init_inventory, supply_optimisation_paths[:,:tau], order_paths, demand_optimisation_paths[:,:tau], spoil_probabilities, spoilage_optimisation_paths[:,:,:tau], tau, h, v, b)
    theta_star = np.log(new_orders)    
    res = minimize(calculate_order_costs, x0=theta_star, args=(number_of_optimisation_paths, inventory_start, supply_optimisation_paths[:,tau:], demand_optimisation_paths[:,tau:], spoil_probabilities, spoilage_optimisation_paths[:,:,tau:], nu, rho, h, v, b), method='Nelder-Mead', tol=1e-9)
        
    return res.x[0]

In [46]:
def optimise_order_decision_numerically(new_orders, number_of_optimisation_paths, inventory, supply_optimisation, demand_optimisation_paths, spoilage_optimisation_paths, spoil_probabilities, order_paths, tau, nu, rho, h, v, b):
    
     return np.round(np.exp(optimise_order_decision(new_orders, number_of_optimisation_paths, inventory, supply_optimisation, demand_optimisation_paths, spoilage_optimisation_paths, spoil_probabilities, order_paths, tau, nu, rho, h, v, b)))


In [47]:
def simulate_system(number_of_simulation_periods, pi, pi_stat, beta1, beta2, number_of_optimisation_paths, tau, nu, rho, type_last_period, supply_is_deterministic, init_inventory, demand_optimisation, spoil_probabilities, spoilage_optimisation, demand_evaluation, spoilage_evaluation, supply_evaluation, orders, new_orders, seed_start, h, v, b):
    total_cost = 0
    inventory_evaluation = init_inventory
    time_before = np.datetime64('now')
    cost_output = 0
    inventory_output = 0
    for t in range(number_of_simulation_periods-nu):
        cost = 0
        if t < number_of_simulation_periods-tau-nu:
            set_numpy_random_seed(t)
            seed = seed_start*t
            inventory = inventory_evaluation.copy()
            supply_optimisation = generate_supply_data_optimisation(pi, pi_stat, beta1, beta2, number_of_optimisation_paths, tau, nu, type_last_period, seed, supply_is_deterministic)
            order_append = optimise_order_decision_numerically(new_orders, number_of_optimisation_paths, inventory, supply_optimisation, demand_optimisation[:,t:t+tau+nu+1], spoilage_optimisation[:,:,t:t+tau+nu+1], spoil_probabilities, orders[t:t+tau], tau, nu, rho, h, v, b)
            orders = np.append(orders, order_append)
            if supply_evaluation[t] == 0:
                type_last_period = 0
            elif supply_evaluation[t] == 1:
                type_last_period = 1
            else:
                type_last_period = 2
            
        inventory_evaluation, cost = period_transition(inventory_evaluation, supply_evaluation[t], orders[t], demand_evaluation[t], spoil_probabilities, spoilage_evaluation[:,t], h, v, b)
        cost_output = np.append(cost_output, cost)
        inventory_output = np.append(inventory_output, np.sum(inventory_evaluation))
        
        if t >= tau:
            total_cost += cost
            
        if (t+nu+1) % 1000 == 0 and t>0:
            print(t)
            print(np.datetime64('now')-time_before)
    
    return orders, total_cost/(number_of_simulation_periods-tau-nu), cost_output, inventory_output

In [52]:
h = 1 # spoilage costs per unit of product
b = 5 # underage costs  per unit of product
v = 0.1 # inventory costs per unit

lambda_mu = 100 # Average demand
lambda_omega = 300 # Parameter for variance of demand
demand_is_deterministic = True

pi = np.transpose(np.array([[0.99, 0.5, 0.5], [0.005, 0.4, 0.1], [0.005, 0.1, 0.4]])) # Transition probability matrix
pi_stat = np.linalg.solve(np.transpose(np.diag(np.array([1,1,1]))-pi+1),np.array([1,1,1])) # Stationary distribution of supply
beta1 = 2 # parameters of beta-dist
beta2 = 3 # parameters of beta-dist
supply_is_deterministic = True

spoil_probabilities = np.array([0.05, 0.1, 0.15, 0.35, 0.2, 0.15]) # Spoilage probabilities
spoilage_is_deterministic = True

number_of_simulation_periods = 200
number_of_optimisation_paths = 10

tau = 3
nu = 3
rho = 0.9

init_inventory = np.full(len(spoil_probabilities), 0, dtype=int)
orders = np.full(tau, lambda_mu+np.sqrt(lambda_mu + lambda_omega))
new_orders = np.full(nu+1, lambda_mu+np.sqrt(lambda_mu + lambda_omega))

seed = 321
set_numpy_random_seed(seed)
type_last_period = rand_choice_nb(np.arange(3), pi_stat, seed)
demand_optimisation, demand_evaluation = generate_demand_data(number_of_simulation_periods, number_of_optimisation_paths, lambda_mu, lambda_omega, seed, demand_is_deterministic)
supply_evaluation = generate_supply_data_evaluation(number_of_simulation_periods, pi, beta1, beta2, type_last_period, seed)
spoilage_optimisation = 0
spoilage_optimisation, spoilage_evaluation = generate_spoilage_data(spoil_probabilities, number_of_optimisation_paths, number_of_simulation_periods, seed, spoilage_is_deterministic)

order_result, costs_result, cost_period, inventory_period = simulate_system(number_of_simulation_periods, pi, pi_stat, beta1, beta2, number_of_optimisation_paths, tau, nu, rho, type_last_period, supply_is_deterministic, init_inventory, demand_optimisation, spoil_probabilities, spoilage_optimisation, demand_evaluation, spoilage_evaluation, supply_evaluation, orders, new_orders, seed, h, v, b)

print(np.mean(cost_period))

27.43383838383838


In [ ]:
%%time

print("t")